In [156]:
import numpy as np
import pandas as pd
import math as m
import matplotlib.pyplot as plt
import warnings
import scipy
from sklearn.decomposition import PCA

from tqdm import tqdm

from visuals import *
from my_lib import *

In [57]:
warnings.simplefilter('ignore')

plt.rcParams['figure.figsize'] = 10, 10
plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markersize'] = 8
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['axes.labelsize'] = 8

In [58]:
def HankelMatrix(qwe, L):  
    N = qwe.shape[0]
    return scipy.linalg.hankel(qwe[ : N - L + 1], qwe[N - L : N])

def prepare_time_series(_dir, centred = True):
    data = pd.read_csv(_dir, delimiter =';', decimal=',')
    
    frequency = len(data)/(data['time'].values[-1]-data['time'].values[0])
    
    assert 490 < frequency < 510, f'Bad frequency {frequency}'

    _x = ( (data['X_value'].values)**2 + (data['Y_value'].values)**2 + (data['Z_value'].values)**2)**.5
    
    if centred:
        _m = np.mean(_x)
        _x = (_x-_m)
        
    _t = (data['time'].values).astype(float).reshape([-1,])

    _t = np.linspace(0,_t[-1]-_t[0],len(_x))
        
    return _x,_t

# Алгоритм с использованием средней фазовой траектории

In [61]:
x_acc_test, t_test = prepare_time_series('./data/30 sec 2_accm.csv')

x_acc, t = prepare_time_series('./data/long_walk_100_acc.csv')

dt = 460*20
x_acc = x_acc[7220:7220+dt]
fig = go.Figure()
fig.add_scatter(y = x_acc, mode='lines', name='x_acc')
fig.show()

In [62]:
x_acc_test_ = x_acc_test[:25000]*0.8

In [63]:
modif = np.concatenate([np.ones(3000),
                        np.linspace(1,0,500),
                        np.zeros(1500)+.2,
                        np.linspace(0,1,1000),
                        np.ones(6000),
                        np.linspace(1,0,400),
                        np.zeros(4000)+.2,
                        np.linspace(0,1,1000)],
                        axis = 0)

In [64]:
modif = np.concatenate([modif,
                       np.ones(len(x_acc_test_)-len(modif))],
                       axis = 0)

In [65]:
x_acc_test_m = x_acc_test_ *0.8 * modif
fig = go.Figure()
fig.add_scatter(y = x_acc_test_m, mode='lines', name='x_acc')
fig.show()

In [126]:
X = HankelMatrix(x_acc,500)
X_test = HankelMatrix(x_acc_test_m,500)

pca = PCA(n_components = 4)
X_PCA = pca.fit_transform(X)
X_PCA_test = pca.transform(X_test)

In [138]:
####################################################
X_mean = np.zeros_like(X_PCA)[:460]
for i in range(18):
     X_mean = X_mean + X_PCA[460*i:460*(i+1)]
X_mean = X_mean/18

####################################################
X_std = np.zeros_like(X_PCA)[:460]
for i in range(18):
    X_std = X_std + (X_PCA[460*i:460*(i+1)] - X_mean)**2
    
X_std = (X_std/(18-1))**.5

D = 2*X_std.mean()

In [128]:
fig_2 = go.Figure()
fig_2.update_layout(autosize=False, width=700, height=700)
fig_2.add_trace(go.Scatter3d(x=X_PCA[:460,0],
                             y=X_PCA[:460,1],
                             z=X_PCA[:460,2],
                             marker=dict(size=0.1,line=dict(width=0.01)),
                             name = 'X_PCA'
                            )
                )
fig_2.add_trace(go.Scatter3d(x=X_mean[:,0],
                             y=X_mean[:,1],
                             z=X_mean[:,2],
                             marker=dict(size=0.1,line=dict(width=0.01)),
                             name = 'Mean'
                            )
                )
fig_2.layout.template = 'plotly_white'
fig_2.show()

In [159]:
#Checking phase area
for i in tqdm(range(len(X_PCA))):
    s = [j for j in range(len(X_mean)) if dist(X_PCA[i],X_mean[j]) <= D]
    assert len(s) != 0,'Problem' + str(i)

100%|██████████| 8701/8701 [00:29<00:00, 290.32it/s]


In [197]:
def L1(x):
    for i in range(len(x)):
        x[i] = max(0,x[i])
    return x

In [235]:
phase_history = []
x_phase_history = {}
n_steps = 0
step_eps = 2*np.pi * 1e-1
temp = 0


dist = lambda x1,x2: np.sum((x1-x2)**2)**.5

def L1(x):
    for i in range(len(x)):
        x[i] = max(0,x[i])
    return x

def L2(NN, phi):
    ans = 0
    for i in range(len(NN)):
        ans += np.abs(NN[i] - phi)
    return ans

L3 = None
L4 = None

# for i in range(len(X_PCA[:2000])):
for i in tqdm(range(len(X_PCA[:4000]))):
    
    if len(phase_history) == 0:
        dists = [dist(X_PCA[i],X_mean[j]) for j in range(len(X_mean))]
        min_index = dists.index(min(dists))

        current_phi = np.linspace(0,2*np.pi,460)[min_index]
        
        x_phase_history[current_phi] = X_PCA[i]
        phase_history.append(current_phi)
        continue
        
    #ближайшие соседи на средней траектории
    NN_indexes = [j for j in range(len(X_mean)) if dist(X_PCA[i],X_mean[j]) < D]
    NN_phases = [(2*np.pi*(ind/460 + n_steps)) for ind in NN_indexes]
    loss1 = lambda phases: np.sum(L1(phase_history[-1] - phases))
    
    #соседство с такими же фазами из предыстории
    NN_x_phases = [phase_history%(2*np.pi) for phase_history, x_history in x_phase_history.items() \
                   if dist(X_PCA[i],x_history) <= D/2]
    
    loss2 = lambda phases: L2(np.array(NN_x_phases), phases)
    
    #оптимизация
    idx_min = np.argmin(loss1(np.array(NN_phases)) + loss2(np.array(NN_phases)%(2*np.pi)))
    
    current_phi = NN_phases[idx_min]
    
    #проверка на количество шагов
    if 2*np.pi - current_phi%(2*np.pi) <= step_eps:
        if len(phase_history) - temp >100:
            temp = len(phase_history)
            n_steps += 1
        
    #заполнение истории
    x_phase_history[current_phi] = X_PCA[i]
    phase_history.append(current_phi)

100%|██████████| 4000/4000 [01:12<00:00, 55.10it/s]


In [237]:
n_steps

8

In [236]:
fig = go.Figure()
fig.add_scatter(y = np.array(phase_history)%(2*np.pi),x=np.arange(len(phase_history)),
                mode='lines',
                name='phase_history')
fig.show()

***Попытка векторизовать (Работает!)***

In [ ]:
eps = 1
delta = 1e-3
v_distance = lambda X, y: np.sum((X - y[None, :])**2, axis=-1)**.5
L1 = lambda x: np.maximum(0, x)
L2 = lambda x: np.maximum(0, x)
phi_cur_history = [0.]
vec_phi_prediction = []
vec_repeated_phase_idx = []

for i in tqdm(range(L)):
    v_i = X_PCA[i, :]
    
    # Ищем NN в радиусе эпсилон для loss
    v_i_NN = np.arange(L)[(v_distance(X_PCA, v_i) < eps) & np.all(X_PCA != v_i[None, :], axis=-1)]
    N_i = np.arange(T)[v_distance(X_mean, v_i) < (1.1*X_std).max(-1)]

    # TODO Просто условие, чтобы не падало, тоже надо подумать
    if len(N_i) == 0:
        vec_phi_prediction.append(phi_cur_history[-1])
        vec_repeated_phase_idx.append(i)
        continue
        
    Phi_i = 2* N_i / T * np.pi

    loss1 = lambda Phi: np.sum(L1(np.array(phi_cur_history)[:, None] - Phi[None, :]), axis=0)
    
    # Смотрим только тех соседей, у которых уже предсказали фазу 
    predicted_NN_only = v_i_NN[np.isin(v_i_NN, np.arange(len(phi_prediction)))]
    loss2 = lambda Phi: np.sum(L2(delta - np.abs(np.array(phi_prediction)[list(predicted_NN_only)][:, None] - Phi[None, :])), axis=0)
    
    idx_min = np.argmin(loss1(Phi_i) + loss2(Phi_i))
    
    phi_i_hat = Phi_i[idx_min]
            
    # Обнуляем фазу, когда подошли близко к 2pi
    if np.abs(phi_i_hat - 2 * np.pi) < 2*1e-1:
        phi_cur_history = [0.]
    else:
        phi_cur_history.append(phi_i_hat)
    vec_phi_prediction.append(phi_i_hat)